In [34]:
import setting
import sys

sys.path.append('..')
import os
os.getcwd()
os.chdir('..')

import pandas as pd
pd.set_option('display.max_rows' , 500)
pd.set_option('display.max_columns' , 200)


from sample.data_prep.data_extract import Dataset


In [38]:
os.getcwd()

'/Users/pinkoi/Projects/analytics/evaluation_tool'

In [33]:
%reload_ext autoreload
%autoreload 2


# extract data

## exp_data

In [11]:
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' )                

# 先用 sql_query_prepare 組合 sql
# extract_data 用來區分這次要撈的資料類型

data_.sql_query_prepare(extract_data = 'exp_data')
print(data_.sql_query) 
    
# 用 data_extraction 來取得 dataframe
# raw_data = data_.data_extraction(extract_data= 'exp_data')



            select 
                coalesce(nullif(uid,''),beacon) as beacon_uid ,uid , beacon, exp_id, var_id, date(created) as created_date
                from ath_experiment
                where exp_id = 'beq_for_you_abexp'
                -- experiment table 不篩時間，只篩最後的 created
                -- 彈性的時間篩選
                and date(created) <= date '2023-10-1'
            


In [ ]:
# exp_start_datea 可以設定 exp 分組的起始時間
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,exp_start_date = '2023-9-14')                
data_.sql_query_prepare('exp_data')
print(data_.sql_query) 



            select 
                coalesce(nullif(uid,''),beacon) as beacon_uid ,uid , beacon, exp_id, var_id, date(created) as created_date
                from ath_experiment
                where exp_id = 'beq_for_you_abexp'
                -- experiment table 不篩時間，只篩最後的 created
                -- 彈性的時間篩選
                and date(created) between date '2023-9-14' and date '2023-10-1'
            


## raw_data (a/aa, SRM)

In [15]:
# trigger 用 dict 的方式記錄，共有 3 層
# event_name 在最外層，用 or 串接
# tracking param 在中間層用 and 串接，param 如果用 () 包起，其中用 or
# 內層設置 cal: 計算邏輯, value: 設定的值, param 如果是 dict_list 要輸入 dict_key:
# 如果輸入 xxx(event_name):None, 只會篩選 event_name = 'xxx'

data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,exp_start_date = '2023-9-14',
                 device = ['iphone' ,'android'] , 
                 version= '6.23.0' ,
                trigger = {'view_feed': None})


# data_.sql_query_prepare('raw_data')
# print(data_.sql_query) 

# extract_data = raw_data 取 a/aa 的指標，會串接 journey，如果有 trigger,lang 也會一並串接 tracking
raw_data = data_.data_extraction(extract_data= 'raw_data')


In [17]:
# raw_data 會切分 device, all 的 metric ，其中 beacon_uid 重複的 row all_metric 會是相同的
raw_data.head()

,exp_id,var_id,beacon_uid,device,device_avg_view_item,device_avg_duration,device_e2,all_avg_view_item,all_avg_duration,all_e2
0,beq_for_you_abexp,1,ll_riza6iskclwudunovapbyrhkuy,iphone,12.0,1790.5,True,12.0,1790.5,True
1,beq_for_you_abexp,1,fb_10227075614688968,iphone,7.0,129.0,True,7.0,129.0,True
2,beq_for_you_abexp,0,sanna1221,iphone,0.0,130.0,False,0.0,130.0,False
3,beq_for_you_abexp,0,pupss91190,iphone,7.0,561.0,True,7.0,561.0,True
4,beq_for_you_abexp,2,fb_925805514565893,iphone,41.0,2507.0,True,41.0,2507.0,True


In [19]:
# version 可以設定為  {'android':'6.23.0' , 'iphone':'6.23.2'}，用來 input 不同的 version
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,exp_start_date = '2023-9-14',
                 device = ['iphone' ,'android'] , 
                 version= {'android':'6.23.0' , 'iphone':'6.23.2'},
                trigger = {'view_feed': None})




In [61]:
# trigger 可以設置 params 以及 value，其中用 dict 包住 param:value
# 其中 dict 中的 param 默認會用 and 做串連，而用 tuple 包住的默認會用 or 串連

data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,exp_start_date = '2023-9-14',
                 device = ['iphone' ,'android'] , 
                 version= {'android':'6.23.0' , 'iphone':'6.23.2'},
                trigger = {'view_feed': ({'from_screen':'navi_browse' , 'screen_name':'feed'}, {'from_screen':'feed_onboarding'})})
# data_.sql_query_prepare('raw_data')
# print(data_.sql_query) 


# trigger 會被轉化 ->
# ((event_name = 'view_feed' and 
# ((event_params_map['from_screen'].string_value in ('navi_browse') and event_params_map['screen_name'].string_value in ('feed')) 
#  or 
#  (event_params_map['from_screen'].string_value in ('feed_onboarding')))))

In [60]:
# param 中可以為 str，會默認用 in，但也可以設置為 dict {cal:xx , value:xx}

data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,exp_start_date = '2023-9-14',
                 device = ['iphone' ,'android'] , 
                 version= {'android':'6.23.0' , 'iphone':'6.23.2'},
                trigger = {'view_feed': {'from_screen':{'cal':'in','value':['navi_browse' , 'feed_onboarding']} , 'screen_name':'feed'}})


# trigger 會被轉化 ->
# ((event_name = 'view_feed' and 
# ((event_params_map['from_screen'].string_value in ('navi_browse' , 'feed_onboarding') and event_params_map['screen_name'].string_value in ('feed'))))

## metric_data

metric 用 dict 紀錄，裡面主要包含兩大塊 metric_trigger, metric_main，一次只能算一個 metric，可以有多個 metric trigger

metric_trigger 是紀錄 metric 額外的 trigger ，會限制計算的 base 

    可以用 tuple 包不同 source 的 trigger, e.g. adoption e1->e2 trigger adoption,e1
    {metric_trigger:({source:'journey' , param:{funnel_e1:None}} , {source:'tracking' , param:{trigger_map}})

metric_main 是 metric 的計算邏輯，裡面主要包含 metirc 以及 condition

    condition 可以視為 metric 的 where 條件，是 metric 的計算 base e.g. measure_dwell_time 需要 condition 設置 view_item
    metirc 是計算正式的 param e.g. measure_dwell_time 的 metric 設置為 current_timer, view_item_cnt 則 metric 是 view_id
    main 裡如果是 boolean 可以只有 condition，但如果 journey metric 的部分特別會放在 metric 中
    metric_main:{metric_name:xxx , 'source':'tracking' , group_level':'event' ,event_type = 'event_on_screen', method:'boolean/count_distinct',  condition:{} , metric:{}}}

source: 取用的 data source

group_level: 後續要 group 的層級，user, day, event e.g. 平均每日的 session_duration 為 day

method: 底層 group by 計算的邏輯 e.g. 每個 view_event 'max' 的 measure_dwell_time ，其中 boolean, count_distinct 為輸入會特殊處理

event_type: tracking metric 的類型 可設可不設，會用是不是 view 開頭判斷

param: trigger 的 condition 要計算的 event -> 跟 trigger 同架構

condition: metric 的條件 e.g. measure_dwell_time 的是 view_item, shop->item 是 view_item:{from_screen:shop}


In [106]:
# metric 一次僅能算一個，主要由 metric_trigger 以及 metric_main 控制

# e.g. funnel e1->e2
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                trigger = {'view_feed': None},
                metric = {
                    'metric_trigger':{'source':'journey' , 'param':{'funnel_e1':None}},
                    'metric_main':{'metric_name':'e1_e2' , 'source':'journey' , 'group_level':'user' ,'method':'boolean', 
                                'metric':{'funnel_e2':None}}}

)

# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)


e1_e2_data = data_.data_extraction(extract_data= 'metric_data')



In [68]:
# e.g. session_duration per_day
# method 用 sum ，method 的定義為 group by level 當層的 agg function， session_duration 可能一天多筆 (多 device)，所以用 sum agg，後續的層級統一用 avg (會算到 per user)
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                trigger = {'view_feed': None},
                metric = {
                    'metric_main':{'metric_name':'session_duration_per_day' , 'source':'journey' , 'group_level':'day' ,'method':'sum', 
                                'metric':{'session_duration':None}} }                
)



# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)

In [105]:
# 如果 metric 是 journey ，可以直接在 metric 設置閾值算 boolean

data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android'] , 
                 version= '6.23.0' ,
                trigger = {'view_feed': None},
                metric = {
                    'metric_main':{'metric_name':'session_duration_more_than_10min' , 'source':'journey' , 'group_level':'day' ,'method':'boolean', 
                                'metric':{'session_duration':{'cal':'>=' , 'value':'600'}}} }                
)

data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)

In [74]:
# e.g. avg.view_feed 
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                # trigger = {'view_feed': None},
                metric = {
                    'metric_main':{'metric_name':'avg.view_feed' , 'source':'tracking' , 'group_level':'user' ,'method':'count_distinct', 
                                'metric':{'view_feed':'view_id'}}}

)
# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)

print('''                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    event_params_map['view_id'].string_value as "avg.view_feed"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed'))
                    ),
''')

                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    event_params_map['view_id'].string_value as "avg.view_feed"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed'))
                    ),



In [96]:
# e.g. avg.dwell_time_from_feed_per_view 
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                trigger = {'view_feed': None},
                metric = {
                    # 'metric_trigger':{'source':'tracking','param':{'view_feed':None}},
                    'metric_main':{'metric_name':'avg.dwell_time_from_feed_per_view' , 'source':'tracking' , 'group_level':'event' ,'method':'max', 
                                   'condition':{'measure_dwell_time':{'screen_name':'feed'} ,'view_feed':None},
                                'metric':{'measure_dwell_time':'current_timer'}}}

)


# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)


print('''          metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    event_params_map['view_id'].string_value,
                    max((coalesce(event_params_map['current_timer'].int_value , 0))) as "avg.dwell_time_from_feed_per_view"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'measure_dwell_time' and ((event_params_map['screen_name'].string_value in ('feed')))))
                    group by (event_params_map['view_id'].string_value) ,beacon , device ,date(event_timestamp)
                    ),

''')

          metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    event_params_map['view_id'].string_value,
                    max((coalesce(event_params_map['current_timer'].int_value , 0))) as "avg.dwell_time_from_feed_per_view"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'measure_dwell_time' and ((event_params_map['screen_name'].string_value in ('feed')))))
                    group by (event_params_map['view_id'].string_value) ,beacon , device ,date(event_timestamp)
                    ),




In [87]:
# e.g. avg.view_item_from_feed_per_view 
# 如果 group by event 並且 event 是 view_type 就一定要設置 event_trigger ，用來判斷 view ->view 的次數或成效

data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                # trigger = {'view_feed': None},
                metric = {
                    'metric_trigger':{'source':'tracking','param':{'view_feed':None}},
                    'metric_main':{'metric_name':'avg.view_item_from_feed_per_view' , 'source':'tracking' , 'group_level':'event' ,'method':'count_distinct', 
                                   'condition':{'view_item':{'from_screen':'feed'}},
                                'metric':{'view':'view_id'}}}

)


# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)


print("""                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end,
                    count(distinct case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['view_id'].string_value else null end) as "avg.view_item_from_feed_per_view"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed')))))
                    group by (case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end) ,beacon , device ,date(event_timestamp)
                    ),
""")

                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end,
                    count(distinct case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['view_id'].string_value else null end) as "avg.view_item_from_feed_per_view"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'view_

In [95]:
# 也可以設置 boolean，判斷 view_feed->view_item 的 adoption，同樣需要設置 metric_trigger，並且 boolean 可以不用 input metric ，直接用 condition 判斷
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                # trigger = {'view_feed': None},
                metric = {
                    'metric_trigger':{'source':'tracking','param':{'view_feed':None}},
                    'metric_main':{'metric_name':'view_feed -> item' , 'source':'tracking' , 'group_level':'event' ,'method':'boolean', 
                                   'condition':{'view_item':{'from_screen':'feed'}}
                                }}

)

# data_.sql_query_prepare(extract_data= 'metric_data')
# print(data_.sql_query)



print('''                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end,
                    coalesce(max(cast(((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) as int)), 0) as "view_feed -> item"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed')))))
                    group by (case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end) ,beacon , device ,date(event_timestamp)
                    ),
''')

                    metric_final as (
                    select beacon , device, date(event_timestamp) as date_,
                    case when ((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) then event_params_map['from_view_id'].string_value else event_params_map['view_id'].string_value end,
                    coalesce(max(cast(((event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed'))))) as int)), 0) as "view_feed -> item"
                    from ath_st_client_side_tracking_without_dirt t 
                    where date(event_date) between date '2023-9-13' and date '2023-10-02' -- event_date 要多取一天
                    and date(event_timestamp) between date '2023-9-13' and date '2023-10-1'
                    and device in ('iphone','android')
                    
                    and ((event_name = 'view_feed') or (event_name = 'view_item' and ((event_params_map['from_screen'].string_value in ('feed')))

In [101]:
# group by user 事例
data_ = Dataset('beq_for_you_abexp' , '2023-9-13' , '2023-10-1' ,
                 device = ['iphone' ,'android' ] , 
                 version= '6.23.0' ,
                # trigger = {'view_feed': None},
                metric = {
                    'metric_trigger':{'source':'tracking','param':{'view_feed':None}},
                    'metric_main':{'metric_name':'view_feed -> item' , 'source':'tracking' , 'group_level':'user' ,'method':'boolean', 
                                   'condition':{'view_item':{'from_screen':'feed'}}
                                }}

)

data_.sql_query_prepare(extract_data= 'metric_data')
print(data_.sql_query)



                
        with expt as (
                select 
                    beacon, uid, exp_id, var_id, date(created) as created_date
                from ath_experiment
            where exp_id = 'beq_for_you_abexp'
            and date(created) <= date '2023-10-1'
            )
            , buyer_journey as (
                select 
                    visit_date, 
                    first_session_timestamp,
                    date(at_timezone(first_session_timestamp, 'Asia/Taipei')) as session_date,
                    beacon_uid, 
                    b.beacon as beacon,
                    b.device as device,
                    b.session_duration_sec session_duration,
                    geo,
                    max_member_status,
                    is_seller, 
                    first_session_timestamp,
                    funnel_e1,
                    funnel_e2,
                    funnel_interest,
                    funnel_consideration,
                    add

# Pre Test

## SRM

In [107]:
from sample.data_prep.data_pre_tests import srm_check

srm_check(raw_data , [0,1,2,3] , [20,20,30,30] , device = ['iphone','android' , 'all'])

{'iphone': {'chi2_stat': 1.0737578896261528,
  'p_value': 0.7834125091494782,
  'is_significant': False},
 'android': {'chi2_stat': 2.9352563097664492,
  'p_value': 0.40171547491124693,
  'is_significant': False},
 'all': {'chi2_stat': 1.2362295979139277,
  'p_value': 0.7443287098994678,
  'is_significant': False}}

## AA/A

In [112]:
from sample.data_prep.data_pre_tests import metric_test
temp = metric_test(raw_data , [0,1] , device = ['all' ])
temp

{'all_avg_view_item': {'group_1_mean': 28.673406664509866,
  'group_2_mean': 29.13847643641059,
  'mean_diff': 0.46506977190072263,
  'z_stat': 0.41375264125470496,
  'p_value': 0.6790552706221336,
  'is_significant': False,
  'se': 1.1240285270213586},
 'all_avg_duration': {'group_1_mean': 941.1413641104992,
  'group_2_mean': 915.9531556363119,
  'mean_diff': -25.188208474187377,
  'z_stat': -1.1561623276856363,
  'p_value': 0.24761476583004294,
  'is_significant': False,
  'se': 21.786048438897172},
 'all_e2': {'group_1_mean': 0.7829181494661922,
  'group_2_mean': 0.776307295029051,
  'mean_diff': -0.006610854437141156,
  'z_stat': -0.8871584718450456,
  'p_value': 0.3749935871454233,
  'is_significant': False,
  'se': 0.007451717643401857}}

In [113]:
temp = metric_test(raw_data , [0,1] , device = ['android' , 'iphone'])
temp

{'android_avg_view_item': {'group_1_mean': 40.81418092909536,
  'group_2_mean': 39.204905063291136,
  'mean_diff': -1.6092758658042214,
  'z_stat': -0.46656734648605086,
  'p_value': 0.6408094536790938,
  'is_significant': False,
  'se': 3.4491823697574913},
 'android_avg_duration': {'group_1_mean': 1007.0101853566691,
  'group_2_mean': 945.4242729393294,
  'mean_diff': -61.58591241733973,
  'z_stat': -1.1797256938561684,
  'p_value': 0.23810933131840706,
  'is_significant': False,
  'se': 52.20358659480739},
 'android_e2': {'group_1_mean': 0.8231458842705787,
  'group_2_mean': 0.819620253164557,
  'mean_diff': -0.003525631106021665,
  'z_stat': -0.2295972476854242,
  'p_value': 0.8184047457772938,
  'is_significant': False,
  'se': 0.01535572025171749},
 'iphone_avg_view_item': {'group_1_mean': 25.667003027245208,
  'group_2_mean': 26.558596918085968,
  'mean_diff': 0.8915938908407597,
  'z_stat': 0.808636531457828,
  'p_value': 0.41872424528967334,
  'is_significant': False,
  'se': 

# Metric Test

In [118]:
# merge_aa 可以 merge 0,1 ->1
# metrics_list 要設置要檢測的 metric, 只接受 list

metric_test(e1_e2_data ,[1,3] , device = 'all' ,metrics_list = ['e1_e2'], significant_level = .05 , merge_aa = True)

{'all_e1_e2': {'group_1_mean': 0.7832948708404676,
  'group_2_mean': 0.7728026533996684,
  'mean_diff': -0.0104922174407992,
  'z_stat': -1.9230933365874654,
  'p_value': 0.05446832903995369,
  'is_significant': False,
  'se': 0.005455906502914555}}